In [2]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10


print(tf.__version__)

root_path = './'

models_path = os.path.join(root_path, 'saved_models_new')
if not os.path.exists(models_path):
    os.mkdir(models_path)


tgt_pth = os.path.join(root_path, 'visualize_dcgan-v20')

if not os.path.exists(tgt_pth):
    os.mkdir(tgt_pth)



BATCH_SIZE = 32
img_rows, img_cols, img_depth = (32, 32, 3)
num_classes = 10


# The data, split between train and test sets:
(x_train, y_train), (_, _) = cifar10.load_data()


# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)


x_train = x_train.astype('float32')
x_train = x_train / 127.5 - 1

def rescaleFn(img):
    return img / 127.5 - 1


datagen=ImageDataGenerator()


# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

train_generator = datagen.flow(x_train, batch_size=BATCH_SIZE)

Using TensorFlow backend.


1.13.1


In [19]:
img_shape = (32, 32, 3)
img_len = np.prod(img_shape)
latent_dim = img_len
noise_len = 100
input_dim = noise_len
ngf = 64  # generator
ndf = 16  # discriminator


def build_discriminator():
    return keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=ndf,
            kernel_size=(4, 4),
            strides=2,
            input_shape=img_shape,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 2
        keras.layers.Conv2D(
            filters=ndf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=ndf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 4
        keras.layers.Conv2D(
            filters=1,
            kernel_size=(1, 1),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        
        #output
        keras.layers.Activation(tf.nn.sigmoid),
        
        #flatten
        keras.layers.Flatten()
    ])


def build_generator():
    
    return keras.Sequential([
        # reshape 1d to 3d
        keras.layers.Reshape((10, 10, 1), input_shape=(input_dim,)),
        
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=ngf * 8,
            kernel_size=(5, 5),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=ngf * 4,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=ngf * 8,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=ngf * 16,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 5
        keras.layers.Conv2DTranspose(
            filters=ngf * 32,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 6
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(2, 2),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])


discriminator = build_discriminator()
generator = build_generator()

generator.summary()
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_15 (Reshape)         (None, 10, 10, 1)         0         
_________________________________________________________________
conv2d_transpose_55 (Conv2DT (None, 23, 23, 512)       12800     
_________________________________________________________________
batch_normalization_75 (Batc (None, 23, 23, 512)       2048      
_________________________________________________________________
activation_123 (Activation)  (None, 23, 23, 512)       0         
_________________________________________________________________
conv2d_transpose_56 (Conv2DT (None, 25, 25, 256)       1179648   
_________________________________________________________________
batch_normalization_76 (Batc (None, 25, 25, 256)       1024      
_________________________________________________________________
activation_124 (Activation)  (None, 25, 25, 256)       0         
__________

In [ ]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator
LR = 0.0002


optim = keras.optimizers.Adam(LR, beta_1=0.5)


discriminator.compile(
    optimizer=optim,
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

# noise
z = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
x = generator(z)
out = discriminator(x)

# GAN
gan = keras.models.Model(inputs=z, outputs=out)

gan.compile(
    optimizer=optim,
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [ ]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 5, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    0.5 * z_real[r_real * 5 + col] + 0.5
                )
            else:
                cell.imshow(
                    0.5 * z_fake[r_fake * 5 + col] + 0.5
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

In [ ]:
EPOCHS = 10251
STEPS = 1


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


# to be visualized
vis_noise = np.random.normal(size=(10, noise_len,))

# fake and real label ( Label Smoothing & Label Flipping )
y_fake = np.zeros(shape=(BATCH_SIZE,))


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        z = np.random.normal(loc=0.0, scale=0.2, size=(BATCH_SIZE, noise_len))
        x_fake = generator.predict(z)
        x_real = next(train_generator)
        
        if x_real.shape[0] < BATCH_SIZE:
            x_real = next(train_generator)

        # add noise
        x_real = x_real + np.random.normal(loc=0, scale=0.01, size=x_real.shape)
        x_fake = x_fake + np.random.normal(loc=0, scale=0.01, size=x_fake.shape)
        y_real = np.random.normal(loc=0.95, scale=0.05, size=x_fake.shape)
        
        # train
        loss_2, acc_2 = discriminator.train_on_batch(x_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(x_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 1):

        z = np.random.normal(loc=0.0, scale=0.2, size=(BATCH_SIZE, noise_len))
        y_real = np.random.normal(loc=0.95, scale=0.05, size=x_fake.shape)
        
        # train
        loss_1, acc_1 = gan.train_on_batch(z, y_real)
        
        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 50 == 0:
        ## visualize results
        viz_fake = generator.predict(vis_noise)
        x_real = next(train_generator)
        visualizeGAN(e, x_real, viz_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();